In [1]:
from html import unescape
from bs4 import BeautifulSoup
import re
import requests
import time
import html
import collections
import pymorphy2
import nltk
from nltk.corpus import stopwords
from fake_useragent import UserAgent

russian_stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()
ua = UserAgent(verify_ssl=False)
session = requests.session()

In [51]:
#функция которая загружает рецензии со страницы
def getPage(url): 
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    review_page = soup.find_all('span', {'itemprop': 'reviewBody'})
    return review_page

#очищает текст от лишних знаков
def cleanText(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

#проверяет сами рецензии отрицательные они или положительные
def detectReviews(reviews):
    cleaned = getCleaned(reviews)
    review = []
    for c in cleaned:
        words = []
        good = 0
        bad = 0
        for word in nltk.word_tokenize(c.lower()):
            if word.isalpha() and word not in russian_stopwords:
                new = morph.parse(word)[0]
                words.append(new.normal_form)   
        for i in words:
            if i in freqGood:
                good += 1
            if i in freqBad:
                bad += 1
        if good >= bad:
            print(c[-100:] + ' ПОЛОЖИТЕЛЬНЫЙ ОТЗЫВ')
            review.append('положит')
        else:
            print(c[-100:] + ' ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ')
            review.append('отриц')
    return review
    
#функция которая очищает все рецензии от лишних знаков и переводит их в формат списка           
def getCleaned(reviews):
    reviews_cleaned = []
    for review in reviews:
        cleaned = cleanText(str(review))
        reviews_cleaned.append(cleaned)
    return reviews_cleaned

#функция составляющая списки слов которые встречаются только в положительных/отрицательных рецензиях.
def getWords(words, antiWords):
    c = collections.Counter()
    new_words = []
    freq_words = []
    for word in words:
        if word not in wordsGood:
            new_words.append(word)
    for word in words:
        c[word] += 1
    for key, value in c.items():
        if value >= 3 and key not in antiWords:
            freq_words.append(key)
    return freq_words

#токенизирует все и приводит в начальную форму   
def tokenizeAndNormalForm(cleaned):
    words = []
    for review in cleaned:
        for i in nltk.word_tokenize(review.lower()):
            if i.isalpha() and i not in russian_stopwords:
                word = morph.parse(i)[0]
                words.append(word.normal_form)
    return words       

In [3]:
#получим 50 отрицательных и положительных рецензий на ла ла ленд и просто 10 рецензий на новый сезон смешариков.
bad_reviews = getPage('https://www.kinopoisk.ru/film/841081/reviews/ord/rating/status/bad/perpage/50/page/1/')
good_reviews = getPage('https://www.kinopoisk.ru/film/841081/reviews/ord/rating/status/good/perpage/50/page/2/')
reviews = getPage('https://www.kinopoisk.ru/film/1379016/reviews/')
print(bad_reviews)
print(good_reviews)
print(reviews)

[<span class="_reachbanner_" itemprop="reviewBody">Отправлялась в зрительный зал, начитавшись и наслушавшись восторженных отзывов друзей из жизни и незнакомцев из интернета. Наиболее часто звучали слова 'искренний', 'волшебный', 'потрясающий'. Возможно, дело было в завышенных ожиданиях, но... <i>я ужасно разочарована.</i> <br/>
<br/>
Хочу сразу оговориться, что песни и танцы действительно чудесны, прекрасно прописаны и поставлены - к этому претензий нет. 'Мюзикловость' картины выдержана более чем удачно.<br/>
<br/>
Но всё остальное!<br/>
<br/>
<b>Сюжет.</b> Поправьте меня, если это не так, но разве мы не сотни раз видели эту пластиковую историю стандартного бульварного романа? Здесь и превращение ругающихся ребят в самую сладкую и идиллическую пару, и стоящий на пути к мечте пустой кошелёк, и сомнения в доступности этой самой мечты, и неоправданные ожидания. Буквально каждую минуту фильма я знала, что произойдёт дальше - и за два часа, что длится картина, успела изрядно заскучать. <br/

In [4]:
cleanedGood = getCleaned(good_reviews)
cleanedBad = getCleaned(bad_reviews)

In [5]:
wordsGood = tokenizeAndNormalForm(cleanedGood)
wordsBad = tokenizeAndNormalForm(cleanedBad)

In [6]:
freqGood = getWords(wordsGood, wordsBad)
freqBad = getWords(wordsBad, wordsGood)

In [33]:
detectReviews(reviews)

ему скромному мнению мультсериал отличный, несмотря на минусы. Рекомендую к просмотру.

У меня всё. ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
ики грустные, это жизнь такая.

P.S. Анатолий Валентинович, спасибо вам за все, мягких вам облаков. ПОЛОЖИТЕЛЬНЫЙ ОТЗЫВ
е убить этот легендарный мультсериал.

Эх...

Один балл за серию с подкроватным монстром.

1 из 10 ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
 где раскроют отношение между Карычем и Совуньей.

2. Развить мерч.

*УДАЧИ С ПРОКТОМ!

 10 из 10 ПОЛОЖИТЕЛЬНЫЙ ОТЗЫВ
ть Смешарики и всё что с ними связано. 

И данная серия, как дань этому следу, такая же прекрасная. ПОЛОЖИТЕЛЬНЫЙ ОТЗЫВ
ею мультфильма, заложенную в далёком 2003. Всей душой хочется в это верить! 

 Спасибо за внимание. ПОЛОЖИТЕЛЬНЫЙ ОТЗЫВ
и» скатятся ещё сильнее. Так что, пока не поздно, ставлю пятёрку первым двенадцати сериям.

5 из 10 ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
ей, ничего более.

3 из 10. Один за попытку, один за мои тёплые воспоминания и один за мимикрятора. ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
тра. Спасибо вам. Спасибо за полтора часа д

['отриц',
 'положит',
 'отриц',
 'положит',
 'положит',
 'положит',
 'отриц',
 'отриц',
 'отриц',
 'положит']

In [52]:
#тут проверяем accuracy
from sklearn.metrics import accuracy_score
results = []
gold = []

for r in bad_reviews:
    gold.append('отриц')
for r in good_reviews:
    gold.append('положит')
    
results = detectReviews(bad_reviews) + detectReviews(good_reviews)


ского следования мечте была для меня очевидна и без просмотра - возможно, поэтому я и не впечатлена. ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
да. Прошу вас, смотрите глубже и не покупайтесь на яркую мишуру. Пусть и очень высокохудожественную. ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
тов в своём жанре и сравнивать из современников почти не с кем. А на безрыбье и рак рыба.

8,5 из 10 ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
 не трезво воспринимаешь критику? Бросай всё и уезжай, может приедет кто-нибудь и спасет твою мечту. ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
екомендую, разве что в качестве фона, пока вы занимаетесь куда более полезными и интересными делами. ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
. Это похвально. Но планка оказалась не так высока, как хотелось бы для такого распиаренного фильма. ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
еуместными.

   В общем, советовать фильм я бы никому не стала, потому что он попросту неинтересен. ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
але, я поймал себя на мысли, что думаю: 'когда же наконец все это кончится и я спокойно смогу уйти?' ОТРИЦАТЕЛЬНЫЙ ОТЗЫВ
ы, это всего лишь средний фильм н

In [56]:
print("Accuracy: %.4f" % accuracy_score(results, gold))

Accuracy: 1.0000


Если говорить про улучшения, то, во-первых, неплохо было бы существенно увеличить объем данных, на которых программа обучается, так как рецензии достаточно большие по объему (в таком случае можно будет еще и оставить слова, которые встречаются больше не двух, а, например, пяти раз). Во-вторых, рецензии скачиваются с сайта КиноПоиск, который при множестве запросов включает капчу. Это можно обойти с помощью selenium'а. Помимо этого, нынешний код иногда нелогичный и содержит "костыли". Например, два раза написан код в котором слова приводятся к начальной форме (в функции определяющей тип рецензии и функции отдельно созданной для этого). Можно улучшить получение текста рецензий и парсить не одна страницу, а написать краулер, который бы собирал рецензии со всех страниц. Также, насколько я знаю, у КиноПоиска есть API и нужно посмотреть есть ли там поиск по названию и если есть, то можно автоматизировать программу, что не надо будет вводить ссылки, а просто вбивать названия.